In [16]:
import pandas as pd
import re

In [17]:
hos_df = pd.read_csv('abb_hos.csv', encoding='utf-8')

In [35]:
hos = []

for i in range(len(hos_df)):
    dropna = hos_df.iloc[i].dropna()
    
    for j in range(len(dropna)):
        if '병원' in dropna[j]:
            dropna[j] = re.sub('병원', '', dropna[j])
            hos.append(dropna[j])
        if '대학교' in dropna[j]:
            a = re.sub('(?m)학교.*$', '', dropna[j]) # '대학교' to end of the line
            hos.append(a)
            b = re.sub('대', '', a)
            hos.append(b)
        else:
            hos.append(dropna[j])

hos = list(set(hos))

In [36]:
# To add transform_regex.py

'|'.join(hos)

'건국|충북대|양산부산|CMC|단국대|계명대|가톨릭대학교성모|계명|KUMC|KHUH|본원|성모|DAMC|서울대|고려대|서울대학교 강남센터|DKUH|대구가톨릭대|전남|동아|이화여자|경상|KBSMS|건국대학교|고려대학교|연세대|중앙대|경북|울산대학교|단국대학교의과대학부속|순천향|중앙|아주|아주대|SNUH|GNUH|서울|전북|원광대학교|충북|AMC|강북삼성|칠곡경북|CBNUH|UUH|경북대|SMC|연건|원광|보라매|대구가톨릭대학교|신촌세브란스|전북대|전남대학교|울산|충남|CNUH|이화여자대학교의과대학|울산대|삼성서울|가톨릭대|영남대|영남대학교|순천향대학교|양산부산대|YUMC|한양|KNUCH|분당서울대|경상대|구가톨릭|JBUH|강남센터|AJOUMC|순천향대|SCHMC|영남|칠곡경북대학교|양산부산대학교|분당서울대학교|충남대|계명대학교동산|동아대|가톨릭|중앙대학교|한양대학교|WKUH|경상대학교|분당서울|아산|KNUH|고려|삼성|AJOU|강남세브란스|경북대학교|부산|PNUH|이화여자대|전북대학교|건국대|서울아산|SNUBH|PNUYH|전남대|연세대학교세브란스|충북대학교|단국|한양대|EUMC|경희대학교|CAUMC|KUH|부산대|HYUMC|경희|DSMC|DCMC|원광대|칠곡경북대|아주대학교|충남대학교|동아대학교|부산대학교|연세|경희대|서울대학교'

In [20]:
import copy

In [21]:
dat = pd.read_csv('phis.csv', encoding='utf-8')

In [22]:
hos_phis = {}
hos_phis_none = {}

for i in range(len(dat)):
    if pd.isnull(dat['의료기관명'][i]):
        hos_phis_none[dat['NOTE_ID'][i]] = dat['note_text'][i]
    else:
        hos_phis[dat['NOTE_ID'][i]] = dat['note_text'][i]

In [23]:
save_origins = copy.deepcopy(hos_phis)

### Policy
1. ~병원이라고 이름붙여진 것은 regex로 처리(`dropna[j] = re.sub('병원', '', dropna[j])`).
2. ~병원으로 이름붙여지지 않는(예. 가톨릭) 것을 vocab으로 저장한다(transform_regex.py `"""+hos_kor+r"""`).

### Call and pre-compile regex
- 기존과는 용도가 약간 틀리지만 organizations 폴더에 넣는다.

In [54]:
import os
import re

name_regex_path = './filters/regex/organizations/'
file_list = os.listdir(name_regex_path)

In [55]:
patterns = []

for file in file_list:
    if file.endswith('_transformed.txt'):
        with open(name_regex_path + file) as r:
            patterns.append(re.compile(r.readline()))

### Search non-detected PHIs (Type I error)

In [56]:
non_detected_hos_keys = []

for notes in hos_phis.items():
    phis = False
    
    for p in patterns:
        if p.search(notes[1]) is not None: 
            phis = True
            continue
    
    if phis == False:
        non_detected_hos_keys.append(notes[0])

In [57]:
non_detected_hos_keys

[]

### 기존 regex에 잡히지 않은 목록
- [0] hos_kor에 일일이 추가하는 방향으로 간다.
- [1] add regex
- [2] 과 종류는 필요한 정보일 때도 있어 띄어쓰기 없이 붙어있는 경우만 취급한다. add regex

### Type II error

In [60]:
false_detected_staff_keys = []

for notes in hos_phis_none.items():
    phis = True
    
    for p in patterns:
        if p.search(notes[1]) is None: 
            phis = False
            continue
    
    if phis == True:
        false_detected_staff_keys.append(notes[0])


In [61]:
false_detected_staff_keys

[]

### Convert to asterisk

In [62]:
from collections import defaultdict

In [63]:
def find_span(span_dict, note_dict):
    for n in note_dict.items():
        note_idx = n[0]
        notes = n[1]
        span_list = []

        for p in patterns:
            if p.search(notes) is not None:
                #span_dict[note_idx].append(p.search(notes).span())
                for m in re.finditer(p, notes):
                    span_dict[note_idx].append(m.span())

    return span_dict

In [64]:
hos_span_dict = defaultdict(list)
raw_span = find_span(hos_span_dict, hos_phis)

In [65]:
for phis in raw_span.items():
    for spans in phis[1]:
        hos_phis[phis[0]] = hos_phis[phis[0]][:spans[0]] + '*'*(spans[1]-spans[0]) + hos_phis[phis[0]][spans[1]:]

In [66]:
transformed = pd.DataFrame(columns = ['idx', 'origin', 'transformed'])

In [67]:
idx = []
tra = []
ori = []


for notes in hos_phis.items():
    tra.append(notes[1])

for notes in save_origins.items():
    idx.append(notes[0])
    ori.append(notes[1])

if len(tra)==len(ori):
    for i in range(len(tra)):
        transformed.loc[i] = [idx[i], ori[i], tra[i]]

In [68]:
transformed[:3]

,idx,origin,transformed
0,313315846,[Conclusion]\n\r2012-04-05 outside CT를 판독하였음. ...,[Conclusion]\n\r2012-04-05 outside CT를 판독하였음. ...
1,300192556,[Conclusion]\n\rCategory 2 - Benign finding\r\...,[Conclusion]\n\rCategory 2 - Benign finding\r\...
2,304614564,[Conclusion]\n\rLarge enhancing mass lesion in...,[Conclusion]\n\rLarge enhancing mass lesion in...


In [69]:
transformed.to_csv('transforemd_hospital_phis.csv', index=False)